## Script para extraer resumenes de paquetes interesantes de pcaps

No le veo el sentido a este código, ni siquiera saca los resumenes en el formato que usan

In [4]:
import random
import re

from scapy.all import Raw
from scapy.utils import RawPcapReader, wrpcap

from scapy.layers.l2 import Ether
from scapy.layers.inet import IP, TCP, ICMP

C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:469: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:483: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,


In [28]:
def process_pcap(file_name, protocol):
    print('Opening {}...'.format(file_name))

    count = 0
    interesting_packet_count = 0

    interesting_packets = []
    for (pkt_data, pkt_metadata,) in RawPcapReader(file_name):
        count += 1

        ether_pkt = Ether(pkt_data)
        if 'type' not in ether_pkt.fields:
            # LLC frames will have 'len' instead of 'type'.
            # We disregard those
            continue
        
        if protocol == 'ICMP':
            if ether_pkt.type != 0x0800:
                # disregard non-IPv4 packets
                continue
            ip_pkt = ether_pkt[IP]
            if ip_pkt.proto != 1:
                # Ignore non-TCP packet
                continue
            interesting_packet_count += 1
            interesting_packets.append(ether_pkt)
        
        if protocol == 'ARP':
            if ether_pkt.type != 0x0806:
                # disregard non-ARP packets
                continue
            interesting_packet_count += 1
            interesting_packets.append(ether_pkt)

    print('{} contains {} packets ({} interesting)'.
          format(file_name, count, interesting_packet_count))

    return interesting_packets


In [30]:
packets_scapy = process_pcap("../data/pcap/nuevos/The-Ultimate-PCAP/The_Ultimate_PCAP_v20231218.pcapng", 'ARP')

Opening ../data/pcap/nuevos/The-Ultimate-PCAP/The_Ultimate_PCAP_v20231218.pcapng...
../data/pcap/nuevos/The-Ultimate-PCAP/The_Ultimate_PCAP_v20231218.pcapng contains 40029 packets (84 interesting)


In [32]:
len(packets_scapy)

84

In [36]:
print(str(packets_scapy[0].show(dump=True)))

###[ Ethernet ]### 
  dst       = c2:3c:19:6c:00:01
  src       = c2:3d:19:6c:00:01
  type      = ARP
###[ ARP ]### 
     hwtype    = 0x1
     ptype     = IPv4
     hwlen     = 6
     plen      = 4
     op        = who-has
     hwsrc     = c2:3d:19:6c:00:01
     psrc      = 10.0.0.1
     hwdst     = c2:3c:19:6c:00:01
     pdst      = 10.0.0.2
###[ Padding ]### 
        load      = '\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'



In [39]:
def str_to_packet(s):
    packet = IP()
    s2i_proto = {v:k for k,v in packet.get_field("proto").i2s.items()}
    _,ip_text,payload_text = re.split("\#{3}\[\s*\w*\s*\]\#{3}",s)
    for line in ip_text.splitlines():
        if line == " " or line == "  \\options   \\":
            continue

        try: 
            key,value = line.split("=")
        except ValueError:
            print(line)
            continue
        if key.strip() == "proto":
            packet[IP].proto = s2i_proto[value.strip()]
            continue
        try:
            packet[IP].fields[key.strip()] = int(value.strip(),0)
        except:
            
            packet[IP].fields[key.strip()] = value.strip()
    # Handle ICMP Payload 
    if packet[IP].proto == 1:
        packet = packet / ICMP()
        type_conversion = {"echo-reply" : 0, "echo-request":8}
        for line in payload_text.splitlines():
            if line == " " or line == "  \\options   \\":
                continue
                
            try: 
                key,value = line.split("=")
            except ValueError:
                print(line)
                continue
            
            try:
                packet[ICMP].fields[key.strip()] = int(value.strip(),0)
            except:
                packet[ICMP].fields[key.strip()] = value.strip()

        if packet.unused == "''":
            packet.unused = b''
        packet.type = type_conversion[packet.type]
    return packet

In [40]:
gen_pkts = [str_to_packet(response.choices[0].text) for response in responses]

def delete_wrong_fields(pkt):

    del pkt[IP].flags
    del pkt[IP].chksum
    del pkt[ICMP].chksum
    
    return pkt
    
gen_pkts = list(map(delete_wrong_fields,gen_pkts))
gen_pkts[0].show2()

with open("generated_icmp.pcap", "wb") as f:
    wrpcap(f, gen_pkts)






  









###[ IP ]### 
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 84
  id        = 25911
  flags     = 
  frag      = 0
  ttl       = 109
  proto     = icmp
  chksum    = 0x8a90
  src       = 8.8.8.8
  dst       = 130.231.202.234
  \options   \
###[ ICMP ]### 
     type      = echo-reply
     code      = 0
     chksum    = 0xfefb
     id        = 0x2
     seq       = 0x102
     unused    = ''



In [41]:
len(gen_pkts)

100

In [24]:
print(responses[0].choices[0].text)



###[ IP ]###
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 84
  id        = 22238
  flags     = DF
  frag      = 0
  ttl       = 64
  proto     = icmp
  chksum    = None
  src       = 130.231.202.234
  dst       = 8.8.8.8
  \options   \
###[ ICMP ]###
     type      = echo-request
     code      = 0
     chksum    = None
     id        = 0x2
     seq       = 0xb2
     unused    = ''


In [173]:
summary,packet = random.choice(packets)

In [42]:
summary

'76.76.2.0 → 130.231.202.234 ICMP 98 Echo (ping) reply id=0x0005, seq=37/9472, ttl=48 (request in 471)'

In [160]:
packet.show()

###[ Ethernet ]### 
  dst       = 84:a9:38:6a:7f:0d
  src       = 70:db:98:81:93:40
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 84
     id        = 53490
     flags     = 
     frag      = 0
     ttl       = 53
     proto     = icmp
     chksum    = 0x54d3
     src       = 9.9.9.9
     dst       = 130.231.202.234
     \options   \
###[ ICMP ]### 
        type      = echo-reply
        code      = 0
        chksum    = 0x3c6
        id        = 0x4
        seq       = 0x101
        unused    = ''
###[ Raw ]### 
           load      = '\\xd5I\\x87c\x00\x00\x00\x00Ӵ\x0c\x00\x00\x00\x00\x00\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f !"#$%&\'()*+,-./01234567'



In [156]:
type(gen_pkts[0][ICMP].unused) == type(packet[ICMP].unused)

True

In [167]:
gen_pkts[2][IP].flags

<Flag 0 ()>

In [168]:
packet[IP].flags

<Flag 0 ()>

In [112]:
gen_pkts[0].show()

###[ IP ]### 
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 84
  id        = 49001
  flags     = DF
  frag      = 0
  ttl       = 64
  proto     = icmp
  chksum    = None
  src       = 130.231.202.234
  dst       = 8.8.8.8
  \options   \
###[ ICMP ]### 
     type      = echo-request
     code      = 0
     chksum    = None
     id        = 0x2
     seq       = 0xb8
     unused    = ''



In [186]:
summary,packet2 = random.choice(packets)
packet2[IP].show()
packet2[IP].flags = "DF"
del packet2[Raw]
del packet2[IP].chksum
del packet2[ICMP].chksum
packet2[IP].show()


###[ IP ]### 
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 84
  id        = 14963
  flags     = 
  frag      = 0
  ttl       = 49
  proto     = icmp
  chksum    = 0xfb5e
  src       = 4.2.2.4
  dst       = 130.231.202.234
  \options   \
###[ ICMP ]### 
     type      = echo-reply
     code      = 0
     chksum    = 0xf79f
     id        = 0x3
     seq       = 0x62
     unused    = ''
###[ Raw ]### 
        load      = '"I\\x87c\x00\x00\x00\x00\\x96{\t\x00\x00\x00\x00\x00\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f !"#$%&\'()*+,-./01234567'

###[ IP ]### 
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 84
  id        = 14963
  flags     = DF
  frag      = 0
  ttl       = 49
  proto     = icmp
  chksum    = None
  src       = 4.2.2.4
  dst       = 130.231.202.234
  \options   \
###[ ICMP ]### 
     type      = echo-reply
     code      = 0
     chksum    = None
     id        = 0x3
     seq       = 0x62
     unused    = ''



In [184]:
packet2[IP].flags

<Flag 2 (DF)>

In [187]:
with open("generated_icmp.pcap", "wb") as f:
    wrpcap(f, packet2)
